In [1]:
import tensorflow as tf
import tensorflow.keras as k
import tensorflow.keras.layers as l

# Table of Contents
- [1-Writing ResNet50 architecture](#01)
- [2-Validation with tensorflow buildin ResNet50](#02)
- [3-Writing ResNet50V1.5 architecture](#03)
- [4-Writing ResNet50V2 architecture](#04)
- [5-Validation with tensorflow buildin ResNet50V2](#05)
- [6-ResNet50 and ResNet50V2 side by side comparison picture](#06)

# 1-Writing ResNet50 architecture<a id="01"></a>

![resnet50 architecture](../img/resnet50v1.PNG)

In [2]:
def residual_block(x, filters, kernel_size=3, stride=1, conv_shortcut=False, name=None):
    """
    A residual block as described in the paper "Deep Residual Learning for Image Recognition"
    conv_shortcut: use convolutional shortcut if True, identity shortcut if False
    """
    
    # assuming image_data_format() is "channels_last" i.e. batch_size, h, w, c
    bn_axis = 3
    
    if conv_shortcut:
        shortcut = l.Conv2D(4 * filters, 1, strides=stride, name=name + "conv0")(x)
        shortcut = l.BatchNormalization(axis=bn_axis, name=name + 'bn0')(shortcut)
    else:
        shortcut = x

    x = l.Conv2D(filters, 1, strides=stride, name=name + "conv1")(x)
    x = l.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + "bn1")(x)
    x = l.Activation("relu", name=name + "relu1")(x)

    # x = l.ZeroPadding2D(padding=1, name=name + "pad2")(x)
    # x = l.Conv2D(filters, kernel_size, strides=1, name=name + "conv2",)(x)
    # or
    x = l.Conv2D(filters, kernel_size, strides=1, padding='same', name=name + "conv2",)(x)
    x = l.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + "bn2")(x)
    x = l.Activation("relu", name=name + "relu2")(x)

    x = l.Conv2D(4 * filters, 1, name=name + "conv3")(x)
    x = l.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + "bn3")(x)
    
    x = l.Add(name=name + "out")([shortcut, x])
    x = l.Activation("relu", name=name + "relu3")(x)
    return x

def ResNet50(input_shape=(224, 224, 3), classes=1000):
    X_input = l.Input(shape=input_shape)
    X = l.ZeroPadding2D(padding=3, name="conv1_pad")(X_input)
    X = l.Conv2D(64, 7, strides=2, name='conv1')(X)
    X = l.BatchNormalization(axis=3, name='conv1_bn')(X)
    X = l.Activation('relu',name='conv1_relu')(X)
    X = l.ZeroPadding2D(padding=1, name="pool1_pad")(X)
    X = l.MaxPooling2D(3, strides=2, name='poo11')(X)

    X = residual_block(X, filters=64, stride=1, conv_shortcut=True, name='g1_b1_conv_shortcut/')
    X = residual_block(X, filters=64, stride=1, conv_shortcut=False, name='g1_b2_id_shortcut/')
    X = residual_block(X, filters=64, stride=1, conv_shortcut=False, name='g1_b3_id_shortcut/')

    X = residual_block(X, filters=128, stride=2, conv_shortcut=True, name='g2_b1_conv_shortcut/')
    X = residual_block(X, filters=128, stride=1, conv_shortcut=False, name='g2_b2_id_shortcut/')
    X = residual_block(X, filters=128, stride=1, conv_shortcut=False, name='g2_b3_id_shortcut/')
    X = residual_block(X, filters=128, stride=1, conv_shortcut=False, name='g2_b4_id_shortcut/')

    X = residual_block(X, filters=256, stride=2, conv_shortcut=True, name='g3_b1_conv_shortcut/')
    X = residual_block(X, filters=256, stride=1, conv_shortcut=False, name='g3_b2_id_shortcut/')
    X = residual_block(X, filters=256, stride=1, conv_shortcut=False, name='g3_b3_id_shortcut/')
    X = residual_block(X, filters=256, stride=1, conv_shortcut=False, name='g3_b4_id_shortcut/')
    X = residual_block(X, filters=256, stride=1, conv_shortcut=False, name='g3_b5_id_shortcut/')
    X = residual_block(X, filters=256, stride=1, conv_shortcut=False, name='g3_b6_id_shortcut/')

    X = residual_block(X, filters=512, stride=2, conv_shortcut=True, name='g4_b1_conv_shortcut/')
    X = residual_block(X, filters=512, stride=1, conv_shortcut=False, name='g4_b2_id_shortcut/')
    X = residual_block(X, filters=512, stride=1, conv_shortcut=False, name='g4_b3_id_shortcut/')

    X = l.GlobalAveragePooling2D()(X)
    X = l.Dense(classes, activation='softmax', name='fc' + str(classes))(X)
    model = k.models.Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [3]:
model = ResNet50(input_shape=(224, 224, 3), classes=1000)
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [4]:
# tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

# 2-Validation with tensorflow buildin ResNet50<a id="02"></a>

In [5]:
validate_model = k.applications.ResNet50(weights=None)
validate_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

# 3-Writing ResNet50v1.5 architecture<a id="03"></a>

![resnet50 architecture](../img/resnetv1_5.png)

In [6]:
def residual_block(x, filters, kernel_size=3, stride=1, conv_shortcut=False, name=None):
    """
    A residual block as described in the paper "Deep Residual Learning for Image Recognition"
    conv_shortcut: use convolutional shortcut if True, identity shortcut if False
    """
    
    # assuming image_data_format() is "channels_last" i.e. batch_size, h, w, c
    bn_axis = 3
    
    if conv_shortcut:
        shortcut = l.Conv2D(4 * filters, 1, strides=stride, name=name + "conv0")(x)
        shortcut = l.BatchNormalization(axis=bn_axis, name=name + 'bn0')(shortcut)
    else:
        shortcut = x

    x = l.Conv2D(filters, 1, strides=1, name=name + "conv1")(x)
    x = l.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + "bn1")(x)
    x = l.Activation("relu", name=name + "relu1")(x)

    # x = l.ZeroPadding2D(padding=1, name=name + "pad2")(x)
    # x = l.Conv2D(filters, kernel_size, strides=1, name=name + "conv2",)(x)
    # or
    x = l.Conv2D(filters, kernel_size, strides=stride, padding='same', name=name + "conv2",)(x)
    x = l.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + "bn2")(x)
    x = l.Activation("relu", name=name + "relu2")(x)

    x = l.Conv2D(4 * filters, 1, name=name + "conv3")(x)
    x = l.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + "bn3")(x)
    
    x = l.Add(name=name + "out")([shortcut, x])
    x = l.Activation("relu", name=name + "relu3")(x)
    return x

def ResNet50v1_5(input_shape=(224, 224, 3), classes=1000):
    X_input = l.Input(shape=input_shape)
    X = l.ZeroPadding2D(padding=3, name="conv1_pad")(X_input)
    X = l.Conv2D(64, 7, strides=2, name='conv1')(X)
    X = l.BatchNormalization(axis=3, name='conv1_bn')(X)
    X = l.Activation('relu',name='conv1_relu')(X)
    X = l.ZeroPadding2D(padding=1, name="pool1_pad")(X)
    X = l.MaxPooling2D(3, strides=2, name='poo11')(X)

    X = residual_block(X, filters=64, stride=1, conv_shortcut=True, name='g1_b1_conv_shortcut/')
    X = residual_block(X, filters=64, stride=1, conv_shortcut=False, name='g1_b2_id_shortcut/')
    X = residual_block(X, filters=64, stride=1, conv_shortcut=False, name='g1_b3_id_shortcut/')

    X = residual_block(X, filters=128, stride=2, conv_shortcut=True, name='g2_b1_conv_shortcut/')
    X = residual_block(X, filters=128, stride=1, conv_shortcut=False, name='g2_b2_id_shortcut/')
    X = residual_block(X, filters=128, stride=1, conv_shortcut=False, name='g2_b3_id_shortcut/')
    X = residual_block(X, filters=128, stride=1, conv_shortcut=False, name='g2_b4_id_shortcut/')

    X = residual_block(X, filters=256, stride=2, conv_shortcut=True, name='g3_b1_conv_shortcut/')
    X = residual_block(X, filters=256, stride=1, conv_shortcut=False, name='g3_b2_id_shortcut/')
    X = residual_block(X, filters=256, stride=1, conv_shortcut=False, name='g3_b3_id_shortcut/')
    X = residual_block(X, filters=256, stride=1, conv_shortcut=False, name='g3_b4_id_shortcut/')
    X = residual_block(X, filters=256, stride=1, conv_shortcut=False, name='g3_b5_id_shortcut/')
    X = residual_block(X, filters=256, stride=1, conv_shortcut=False, name='g3_b6_id_shortcut/')

    X = residual_block(X, filters=512, stride=2, conv_shortcut=True, name='g4_b1_conv_shortcut/')
    X = residual_block(X, filters=512, stride=1, conv_shortcut=False, name='g4_b2_id_shortcut/')
    X = residual_block(X, filters=512, stride=1, conv_shortcut=False, name='g4_b3_id_shortcut/')

    X = l.GlobalAveragePooling2D()(X)
    X = l.Dense(classes, activation='softmax', name='fc' + str(classes))(X)
    model = k.models.Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [7]:
modelv1_5 = ResNet50v1_5(input_shape=(224, 224, 3), classes=1000)
modelv1_5.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [8]:
# tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

# 4-Writing ResNet50V2 architecture<a id="04"></a>

![resnet50V2 architecture](../img/resnet50v2.PNG)

In [9]:
def residual_block_v2(x, filters, kernel_size=3, stride=1, conv_shortcut=False, name=None):
    """
    A residual block as described in the paper "Deep Residual Learning for Image Recognition"
    conv_shortcut: use convolutional shortcut if True, else maxpool shortcut if stride>1 else identity shortcut 
    """
    
    # assuming image_data_format() is "channels_last" i.e. batch_size, h, w, c
    bn_axis = 3

    preact = l.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + "preact_bn")(x)
    preact = l.Activation("relu", name=name + "preact_relu")(preact)
    
    if conv_shortcut:
        shortcut = l.Conv2D(4 * filters, 1, strides=stride, name=name + "conv0")(preact)
    else:
        shortcut = (l.MaxPooling2D(1, strides=stride, name=name+'maxpool0')(x) if stride > 1 else x)

    x = l.Conv2D(filters, 1, strides=1, use_bias=False, name=name + "conv1")(preact)
    x = l.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + "bn1")(x)
    x = l.Activation("relu", name=name + "relu1")(x)

    # x = l.ZeroPadding2D(padding=1, name=name + "pad2")(x)
    # x = l.Conv2D(filters, kernel_size, strides=stride, use_bias=False, name=name + "conv2",)(x)
    # or
    x = l.Conv2D(filters, kernel_size, strides=stride, padding='same', use_bias=False, name=name + "conv2")(x)
    x = l.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + "bn2")(x)
    x = l.Activation("relu", name=name + "relu2")(x)

    x = l.Conv2D(4 * filters, 1, name=name + "conv3")(x)
    
    x = l.Add(name=name + "add_out")([shortcut, x])
    return x

def ResNet50v2(input_shape=(224, 224, 3), classes=1000):
    X_input = l.Input(shape=input_shape)
    X = l.ZeroPadding2D(padding=3, name="conv1_pad")(X_input)
    X = l.Conv2D(64, 7, strides=2, name='conv1')(X)
    X = l.ZeroPadding2D(padding=1, name="pool1_pad")(X)
    X = l.MaxPooling2D(3, strides=2, name='poo11')(X)

    X = residual_block_v2(X, filters=64, stride=1, conv_shortcut=True, name='g1_b1_conv_shortcut/')
    X = residual_block_v2(X, filters=64, stride=1, conv_shortcut=False, name='g1_b2_id_shortcut/')
    X = residual_block_v2(X, filters=64, stride=2, conv_shortcut=False, name='g1_b3_maxpool_shortcut/')

    X = residual_block_v2(X, filters=128, stride=1, conv_shortcut=True, name='g2_b1_conv_shortcut/')
    X = residual_block_v2(X, filters=128, stride=1, conv_shortcut=False, name='g2_b2_id_shortcut/')
    X = residual_block_v2(X, filters=128, stride=1, conv_shortcut=False, name='g2_b3_id_shortcut/')
    X = residual_block_v2(X, filters=128, stride=2, conv_shortcut=False, name='g2_b4_maxpool_shortcut/')

    X = residual_block_v2(X, filters=256, stride=1, conv_shortcut=True, name='g3_b1_conv_shortcut/')
    X = residual_block_v2(X, filters=256, stride=1, conv_shortcut=False, name='g3_b2_id_shortcut/')
    X = residual_block_v2(X, filters=256, stride=1, conv_shortcut=False, name='g3_b3_id_shortcut/')
    X = residual_block_v2(X, filters=256, stride=1, conv_shortcut=False, name='g3_b4_id_shortcut/')
    X = residual_block_v2(X, filters=256, stride=1, conv_shortcut=False, name='g3_b5_id_shortcut/')
    X = residual_block_v2(X, filters=256, stride=2, conv_shortcut=False, name='g3_b6_maxpool_shortcut/')

    X = residual_block_v2(X, filters=512, stride=1, conv_shortcut=True, name='g4_b1_conv_shortcut/')
    X = residual_block_v2(X, filters=512, stride=1, conv_shortcut=False, name='g4_b2_id_shortcut/')
    X = residual_block_v2(X, filters=512, stride=1, conv_shortcut=False, name='g4_b3_maxpool_shortcut/')

    X = l.BatchNormalization(axis=3, epsilon=1.001e-5, name="post_bn")(X)
    X = l.Activation("relu", name="post_relu")(X)
    
    X = l.GlobalAveragePooling2D()(X)
    X = l.Dense(classes, activation='softmax', name='fc' + str(classes))(X)
    model = k.models.Model(inputs=X_input, outputs=X, name='ResNet50V2')

    return model

In [10]:
modelv2 = ResNet50v2(input_shape=(224, 224, 3), classes=1000)
modelv2.summary()

Model: "ResNet50V2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                         

In [11]:
# tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

# 5-Validation with tensorflow buildin ResNet50V2<a id="05"></a>

In [12]:
validate_model = k.applications.ResNet50V2(weights=None)
validate_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_5[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                         

# 6-ResNet50 and ResNet50V2 side by side comparison picture<a id="06"></a>

![resnet50s part1 architecture](../img/resnet50comp1.png)
![resnet50s part2 architecture](../img/resnet50comp2.PNG)
![resnet50s part2 architecture](../img/resnet50comp3.PNG)
![resnet50s part2 architecture](../img/resnet50comp4.PNG)